assign、eval、query，分别用于赋值、查询和执行计算。
## 01 assign

In [1]:
import pandas as pd

df=pd.DataFrame([[1,11],[2,22]],columns=['A','B'])
df

,A,B
0,1,11
1,2,22


例如，对于以上简单的DataFrame数据框，需要创建一个新的列C，一般来说可能有3种创建需求：常数列、指定序列数据以及由已知列通过一定计算产生。那么应用assign完成这3个需求分别是：

In [2]:
#赋值常数列
df.assign(C=1)

,A,B,C
0,1,11,1
1,2,22,1


In [4]:
#赋值已有可迭代对象
df.assign(C=[111,222])

,A,B,C
0,1,11,111
1,2,22,222


In [5]:
#根据已有列计算赋值
df.assign(C=df.A+df.B)

,A,B,C
0,1,11,12
1,2,22,24


注意事项：
- 1、assign赋值新列时，一般用新列名=表达式的形式，其中新列名为变量的形式，所以不加引号（加引号时意味着是字符串）；
- 2、assign返回创建了新列的dataframe，所以需要用新的dataframe对象接收返回值；
- 3、assign不仅可用于创建新的列，也可用于更新已有列，此时创建的新列会覆盖原有列。

## 02 eval
实际上，eval是一个Python基础函数，用于执行字符串形式的计算表达式，例如以下简单实例：

In [6]:
eval("1+2+3")

6

那么，eval作为pandas.dataframe数据结构的一个接口，执行功能应该也与执行计算有关。另一方面，pandas中实际上是内置了大量的SQL类语法（包括下面要介绍的query也是），而eval的功能正是执行类似SQL语法中的计算，对已知列执行一定的计算时可用eval完成。例如，仍以前述由A和B列产生C列为例，应用eval的方法为：

In [7]:
df.eval('C=A+B')

,A,B,C
0,1,11,12
1,2,22,24


了解SQL语法的都知道可用@前缀修饰自定义变量，这一用法在这里的eval中也得以保留，此时可非常方便的引用外部变量。当然，eval中的计算表达式本身属于字符串形式，所以自然也可以用Python的通用字符串引用方法。如下图所示。

In [8]:
# @修饰引用变量
A=100
df.eval('C=@A+A')

,A,B,C
0,1,11,101
1,2,22,102


In [9]:
# f字符串前缀引用变量
Z=100
df.eval(f'C={A}+A')

,A,B,C
0,1,11,101
1,2,22,102


注意事项：
- 1、eval支持接收一个inplace参数控制原地创建新变量或者返回新的dataframe；也支持仅用表达式而不设置新变量名，此时返回数据为series格式，如下图所示；
- 2、eval表达式中也支持调用函数执行复杂计算。

In [10]:
#不设置新列名返回series

df.eval('B+A')

0    12
1    24
dtype: int64

In [12]:
df.eval('C=B+A',inplace=True)
df

,A,B,C
0,1,11,12
1,2,22,24


In [15]:
df.eval('D=B+A',inplace=False),df

(   A   B   C   D
 0  1  11  12  12
 1  2  22  24  24,
    A   B   C
 0  1  11  12
 1  2  22  24)

## 03 query
这应该是最近使用最为频繁的一个接口了，pandas中虽然也提供了多种数据筛选方式，例如loc中增加表达式、或者直接用df[df[]……]等等，但总觉得用起来不够优雅，尤其是要写两遍df以及[]等等，此时如果灵活运用query函数，那么会便捷不少。尤其是query也是类似于SQL中where关键字的语法逻辑，用起来会很顺滑。

In [16]:
df

,A,B,C
0,1,11,12
1,2,22,24


In [17]:
#单条件查询
df.query('A==1')

,A,B,C
0,1,11,12


In [18]:
#执行一定计算查询
df.query('A+B==24')

,A,B,C
1,2,22,24


当然，之所以说query中支持类似SQL的语法，是因为其也有两个SQL中标志性的设计，其一是@引用自定义外部变量，其二是对于特殊的列名（例如包含空格的字符）可以用反引号``加以修饰引用。例如，下述例子中C C列中有个空格，直接用于字符串表达式会存在报错，此时可使用反引号加以修饰，同时查询条件中应用了@修饰符引用外部变量。当然，与eval中类似，这里当然也可以用f字符串修饰引用。

In [21]:
df['C C']=[100,200]
df

,A,B,C,C C
0,1,11,12,100
1,2,22,24,200


In [22]:
x=100
df.query('`C C`==@x')

,A,B,C,C C
0,1,11,12,100


注意事项：
- 1. query中也支持inplace参数，控制是否将查询过滤条件作用于dataframe本身；
- 2. 与eval类似，query中也支持引用外部函数。

# 04 apply
使用 apply 将 dataframe 中内容为 list 的列拆分为多列。

In [2]:
!pip install akshare

  Using cached decorator-4.4.2-py2.py3-none-any.whl (9.2 kB)
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.8.2
    Uninstalling beautifulsoup4-4.8.2:
      Successfully uninstalled beautifulsoup4-4.8.2
  Attempting uninstall: decorator
    Found existing installation: decorator 4.4.1
    Uninstalling decorator-4.4.1:
      Successfully uninstalled decorator-4.4.1
  Attempting uninstall: tabulate
    Found existing installation: tabulate 0.8.3
    Uninstalling tabulate-0.8.3:
      Successfully uninstalled tabulate-0.8.3


In [11]:
import pandas as pd
import akshare as ak

In [12]:
print(ak.__version__)
print(pd.__version__)

0.7.56
1.0.1


In [13]:
# 先从 akshare 获取需要的数据，分为两步，第一步是获取基金代码的列表，如下：
df=ak.fund_em_fund_name().head(20).tail(5)
df=df[['基金代码','基金简称']]
print(df)

      基金代码        基金简称
15  000017     财通可持续混合
16  000020  景顺长城品质投资混合
17  000021    华夏优势增长混合
18  000024   大摩双利增强债券A
19  000025   大摩双利增强债券C


In [17]:
ak.fund_em_fund_name().columns

Index(['基金代码', '拼音缩写', '基金简称', '基金类型', '拼音全称'], dtype='object')

In [14]:
# 第二步是获取基金净值数据和净值日期，通过一个自定义函数来获取，自定义函数如下：
# 获取基金单位净值以及净值日期
def get_mutual_fund(code):
    df = ak.fund_em_open_fund_info(fund=code, indicator="单位净值走势")
    df.columns = ['净值日期', '单位净值', 'equityReturn', 'unitMoney']
    df['净值日期'] = pd.to_datetime(df['净值日期'])
    df = df.sort_values('净值日期',ascending=False)
    unit_equity = df.head(1)['单位净值'].values[0]
    date_latest = df.head(1)['净值日期'].values[0]
    return [unit_equity, date_latest]

In [15]:
# 对于这个自定义函数，在 pandas 使用 apply 来应用自定义函数，这是使用 apply 的一种常用的方法，如下
# 获取基金最新的单位净值和净值日期
df['tmp'] = df['基金代码'].apply(get_mutual_fund)
print(df)

ValueError: Length mismatch: Expected axis has 3 elements, new values have 4 elements